In [1]:
import os
import polars as pl
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
# fix and align schemas
cwd = os.getcwd()
dfs = []
for file in os.listdir(cwd):
    if '.xlsx' in file:
        frame = pl.read_excel(
            file,
            schema_overrides={
                'HORA ':pl.String,
                'KILOMETRO ':pl.Float64
            }
        )
        frame = frame.rename(mapping={'TIPO DE VIA':'TIPO DE LA VIA'}, strict=False)
        dfs.append(frame)
df = pl.concat(dfs).lazy()


In [ ]:
df.collect_schema()

In [30]:
q = (
    df
    .unpivot(
        on=[
            'MEDIO DE TRANSPORTE DE LA VICTIMA ',
            'Peatón Atrop. por:',
            'Colisión con: '
        ],
        index=['TOTAL '],
        variable_name='Involucrados',
        value_name='Tipo de vehiculo'
        
    )
    .with_columns(
        pl.col('Involucrados').replace({
            'MEDIO DE TRANSPORTE DE LA VICTIMA ':'Victima',
            'Peatón Atrop. por:':'Peaton',
            'Colisión con: ':'Causante'
        })
    )
    # excluir cuando no hay vehiculo reportado
    .filter(~pl.col('Tipo de vehiculo').is_null())
    .group_by(
        pl.col('Tipo de vehiculo'),
        pl.col('Involucrados')
    )
    .agg(
        pl.col('TOTAL ').sum().alias('Accidentes')
    )
    .with_columns(
        pl
        .col('Accidentes')
        .sum()
        .over(partition_by='Tipo de vehiculo')
        .alias('Total Por Tipo de Vehiculo')
    )
    .sort(
        by=['Total Por Tipo de Vehiculo'],
        descending=True
    )
    .collect()
)

In [31]:
q

Tipo de vehiculo,Involucrados,Accidentes,Total Por Tipo de Vehiculo
str,str,i64,i64
"""MOTOCICLETA""","""Peaton""",29,867
"""MOTOCICLETA""","""Victima""",724,867
"""MOTOCICLETA""","""Causante""",114,867
"""CARRO""","""Peaton""",16,196
"""CARRO""","""Victima""",79,196
…,…,…,…
"""VOLTEO Y AUTOBUS""","""Causante""",1,1
"""CAMION Y CARRO""","""Causante""",1,1
"""MOTOCICLETA Y JEEPETA""","""Causante""",1,1
